# Симплексный метод решения ЗЛП

**Симплекс-метод** – это итеративный процесс направленного решения системы уравнений по шагам, который начинается с опорного решения и в поисках лучшего варианта движется по угловым точкам области допустимого решения, улучшающих значение целевой функции до тех пор, пока целевая функция не достигнет оптимального значения.

## Реализация симплекс-метода

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_base_col_idx(a):
    '''
    Принимает на вход вектор, среди его отрицательных
    элементов ищет максимальный по модулю и возвращает
    его индекс
    '''
    a = np.array(a)
    a[a >= 0] = np.nan
    return np.nanargmax(np.abs(a))

In [3]:
def get_base_row_idx(a):
    '''
    Принимает на вход вектор, среди его положительных
    элементов ищет минимальный и возвращает его индекс
    '''
    a = np.array(a)
    a[a <= 0] = np.nan
    return np.nanargmin(a)

In [4]:
def simplex(M):
    # цикл пока в нижней строке есть отрицательные элементы
    while(len(M[-1][M[-1] < 0]) != 0):
        # из отрицательных коэффициентов нижней строки 
        # выбираем наибольший по абсолютной величине. Индекс
        # этого элемента и будет индексом опорного стоблца 
        base_col_idx = get_base_col_idx(M[-1, :-1])

        # в качестве опорной строки берём строку с наименьшим значением
        # (последний столбец / опорный столбец)
        base_row_idx = get_base_row_idx(M[:-1, -1] / M[:-1, base_col_idx])

        # опорная строка и опорный столбец
        base_col = M[:, base_col_idx]
        base_row = M[base_row_idx]

        # опорный элемент
        base_elem = M[base_row_idx][base_col_idx]

        # пересчёт симплексной таблицы 
        new_M = []
        for i in range(len(M)):
            if (i != base_row_idx):
                new_M.append(M[i] - (base_row * base_col[i]) / base_elem)
            else:
                new_M.append(M[i] / base_elem)

        M = np.array(new_M)

    return M[:, -1]

## Проверка

Задача описывается следующим образом:

$\left\{\begin{matrix}
x_1 + x_2 \leq 550
\\2x_1 + 3x_2 \leq 1200
\\12x_1 + 30x_2 \leq 9600
\end{matrix}\right.$

$F(x_1, x_2) = 3x_1 + 4x_2 \rightarrow max$

Правильный ответ:

$x_1=450, x_2=100$

$F(x_1, x_2) = 3 \cdot 450 + 4 \cdot 100 = 1750$

Составляем симплекс-таблицу:

In [5]:
M = np.array([[1,   1, 1, 0, 0, 550 ],
              [2,   3, 0, 1, 0, 1200],
              [12, 30, 0, 0, 1, 9600],
              [-3, -4, 0, 0, 0, 0   ]], dtype=float)

Результат работы метода:

In [6]:
simplex(M)

array([ 1200.,   450.,   100.,  1750.])

Метод вернул вектор, второй и третий элементы которого соотвествует оптимальным значениям $x_1$ и $x_2$ соответственно. Последний элемент – значение функции $F(x_1, x_2)$. Полученный результат является правильным, что говорит о корректности работы метода.